In [1]:
import pandas as pd
import numpy as np
import tweepy
import json
import warnings
warnings.filterwarnings('ignore')


In [2]:
consumer_key = 'XfcbXSUkdUBxbKZFRI0iwzFta'
consumer_secret = 'JM9FM9kYfpiGkCpfw0vaeUjNoY9VKO1lCBHgFx3gJE83DQ3DB0'
access_token = '1031510125784121344-PyoDNc5LJVhpI4q0yiOeG7WprrnAQJ'
access_token_secret = 'mcvN9YOdGZDdPt4DwO8ZUo5AA8iAXx0D4DHTOPdxURQ1Q'

In [3]:
KeyWord = 'humalog'

In [4]:
FileNameCSV = KeyWord + '_output.csv'
FileNameJSON = KeyWord + 'json.txt'


In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Edit the Keyword 

last_20_tweets = api.search(KeyWord,count = 100)

In [6]:
len(last_20_tweets)

100

In [7]:
my_list_of_dicts = []
for each_json_tweet in last_20_tweets:
    my_list_of_dicts.append(each_json_tweet._json)

In [8]:
with open(FileNameJSON, 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))

In [9]:
my_demo_list = []
with open('tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = each_dictionary['text']
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        created_at = each_dictionary['created_at']
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'created_at': created_at,
                            })
        tweet_json = pd.DataFrame(my_demo_list, columns = 
                                  ['tweet_id', 'text', 
                                   'favorite_count', 'retweet_count', 
                                   'created_at'])

In [10]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())


CleanText =[]
for index, row in tweet_json.iterrows():
    CleanText.append(clean_tweet(row['text']).lower())

tweet_json["CleanText"] = CleanText


In [11]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [12]:
Sentiment =[]
for index, row in tweet_json.iterrows():
    Sentiment.append(get_tweet_sentiment(row['CleanText']))

tweet_json['Sentiment'] = Sentiment   

In [35]:
## WARNING


## If we are adding external file then only run this

import pandas as pd
 
tweet_json = pd.read_csv("/home/ajadhav/Humalog Data _Complete (3).csv",encoding='latin-1')

tweet_json.columns = ['text','Sentiment']


CleanText =[]
for index, row in tweet_json.iterrows():
    CleanText.append(clean_tweet(row['text']).lower())

tweet_json["CleanText"] = CleanText


In [33]:
tweet_json.Sentiment.value_counts()

Positive    3141
Negative    1173
Neutral     1007
Name: Sentiment, dtype: int64

In [36]:
tweet_json.head()

,text,Sentiment,CleanText
0,""" The way to know if the meter itself is okay ...",Positive,the way to know if the meter itself is okay is...
1,""" The way to know if the meter itself is okay ...",Neutral,the way to know if the meter itself is okay is...
2,""" The way to know if the meter itself is okay ...",Positive,the way to know if the meter itself is okay is...
3,"it might have ""burned out"" some of my ë_-cells...",Negative,it might have burned out some of my cells sinc...
4,"it might have ""burned out"" some of my ë_-cells...",Negative,it might have burned out some of my cells sinc...


In [37]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tweet_json, test_size=0.33, random_state=42)

In [38]:
from nltk.tokenize import word_tokenize

X_train = train.CleanText.apply(word_tokenize)
X_test = test.CleanText.apply(word_tokenize)
y_train = train.Sentiment
y_test = test.Sentiment

tweet_json['Tokens'] = tweet_json['CleanText'].apply(word_tokenize)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
tweet_json['Tokens']=[" ".join(review) for review in tweet_json['Tokens'].values]
v.fit(tweet_json['Tokens'])


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [40]:
X_train = X_train.apply(lambda x: ' '.join(x))

X_train = v.transform(X_train)

X_test = X_test.apply(lambda x: ' '.join(x))

X_test = v.transform(X_test)


In [41]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train,y_train )
predicted = clf.predict(X_test)

In [42]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(y_test, predicted))
print(accuracy_score(y_test, predicted))

             precision    recall  f1-score   support

   Negative       0.61      0.93      0.74       400
    Neutral       0.91      0.69      0.79       334
   Positive       0.91      0.80      0.85      1022

avg / total       0.84      0.81      0.81      1756

0.809225512528


In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
print(confusion_matrix(y_test, predicted))

[[370   5  25]
 [ 50 232  52]
 [186  17 819]]


In [45]:
test["PredictedSentiment"] = predicted

In [46]:
output = test[['text','Sentiment','PredictedSentiment']]

In [47]:
output.to_csv(FileNameCSV,index=False)

In [71]:
# How to run this code

# First You need to add the KeyWord for which you want to do Sentiment Analysis
# Suppose I want to search for "Humalog then add this word into Chunk Number 3
# Example
# ---------------
#  api.search('humalog',count = 100)
#----------------

# then rest of the code will pre process the text data means tweet and convert that data into Numeric form

# Then we will apply ML model which is Naive Bayes

# After building the model it will validate the results and create the confusion matrix and Accuracy Score

# It will also save the output in csv format where three columns are there

# Tweet, Actual Sentiment and Predicted Sentiment will be also there

